In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def f(x):
    return 3*x**2 - 4*x + 5

In [ ]:
f(3.0)

In [ ]:
xs = np.arange(-5, 5, 0.25)
xs

In [ ]:
ys = f(xs)
plt.plot(xs, ys)

In [ ]:
# what is the derivative of f(x)?
# not a symbolic approach
# what is the derivative measuring?
# slope after adding a little bit (h)
h = 0.0000000001
x = 2/3
(f(x+h) - f(x))/h

In [ ]:
a = 2.0
b = -3.0
c = 10.0
d = a*b + c
print(d)

In [ ]:
# derivative of d with respect to a, b, and c
h = 0.00001

a = 2.0
b = -3.0
c = 10.0

# d1 = a*b + c
# a += h
# d2 = a*b + c

# print(f'{d1=}')
# print(f'{d2=}')
# print(f'slope={(d2-d1)/h}')


# d1 = a*b + c
# b += h
# d2 = a*b + c

# print(f'{d1=}')
# print(f'{d2=}')
# print(f'slope={(d2-d1)/h}')


d1 = a*b + c
c += h
d2 = a*b + c

print(f'{d1=}')
print(f'{d2=}')
print(f'slope={(d2-d1)/h}')

In [ ]:
# lets build some data structures
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f'Value(data={self.data})'
    
    def __add__(self, other):
        return Value(self.data + other.data, (self, other), '+')
    
    def __mul__(self, other):
        return Value(self.data * other.data, (self, other), '*')

In [ ]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label = 'f')
L = d * f; L.label = 'L'
L

In [ ]:
from graphviz import Digraph

def trace(root):
    # build a set of all notes & edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges


def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # left to right
    
    nodes, edges = trace(root)

    # for any value in the graph create a rectangular 'record' node for it
    for n in nodes:
        uid = str(id(n))
        dot.node(name = uid, label = "{ %s | data %.4f }" % (n.label, n.data), shape='record')
        if n._op:
            # if this value is a result of some operation, create an op node for it
            dot.node(name = uid + n._op, label = n._op)
            # and conect this node to it
            dot.edge(uid + n._op, uid)

    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot

In [ ]:
# visualize forward pass
draw_dot(L)

In [ ]:
# backprop - walk back the graph and calculate the gradient along the path
# compute derivative of node with respect to L
# derivative of output with respect to its leaf nodes